In [1]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA, PCA
from sklearn.cluster import KMeans, Birch
from scipy.optimize import minimize_scalar
from sklearn.manifold import TSNE
from sklearn.datasets import load_breast_cancer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)


import bnlearn as bn
from pgmpy.models import BayesianNetwork
from pgmpy.models import BayesianModel

import os
import logging
try:
    logger.debug('logger is up')
except:
# base logger setup, to standardize logging across classes
    name = 'rce'
    formatter = logging.Formatter(fmt='%(asctime)s -  %(name)s - %(levelname)s  - %(message)s')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
    logger = logging.getLogger(name)
    logger.setLevel(logging.ERROR)
    logger.addHandler(handler)

def get_acc(y_true, y_pred):
    if all((y_true==1, y_pred==1)):
        return 'TP'
    elif all((y_true == 1, y_pred == 0)):
        return 'FN'
    elif all((y_true == 0, y_pred == 1)):
        return 'FP'
    elif all((y_true == 0, y_pred == 0)):
        return 'TN'
    else:
        raise ValueError(F'unknown input y_true: {y_true}  y_pred: {y_pred}')

symbol_map = {'TP':'square', 'FN':'square-x', 'TN': 'circle', 'FP': 'circle-x'}
def get_age(x):
    return str(int(x/10)*10) + 's'


def get_age_group(x, base=5):
    v = int(base * np.trunc(float(x)/base))
    
    return F'Age {v}-{v+base-1}'
    

In [2]:
data_dir = os.environ['CANCER_UPTAKE_DATA_HOME']
data_home = '/'.join(data_dir.split('/')[:-1]) + '/MamoStudy/artifacts/bn_structure_learning/'

path = F'{data_home }/complete_data.csv'
df = pd.read_csv(path).drop(['Unnamed: 0'], axis=1).set_index('pid')



df.loc[:, "Age Group"] = df.AgeAtVisit.apply(get_age_group).astype(str)
df.loc[:, 'Race'] = df.loc[:, 'White-Cauc'].replace({1:'White-Caus', 0: ''}) + \
                    df.loc[:, 'African-American'].replace({1:'African-American', 0: ''})  + \
                    df.loc[:, 'Asian'].replace({1:'', 0: ''})

df.loc[:, 'Race'] = df.loc[:, 'Race'] .replace({'': 'Other or les frequent Race'})




In [3]:
feature_names_all = ['Married',
'Single',
'Divorced_or_Sep',
'Wpidowed',
'ph_cancer',
'ph_cell',
'ph_colon',
'ph_skin',
'ph_thyroid',
'ph_uterine',
'fhneg_breast',
'fhneg_cancer',
'fhneg_colon',
'fhneg_endometrial',
'fhneg_ovarian',
'fhneg_skin',
'fhneg_thyroid',
'fhneg_uterine',
'fh_brain',
'fh_breast',
'fh_cancer',
'fh_cervical',
'fh_colon',
'fh_endometrial',
'fh_esophageal',
'fh_liver',
'fh_lung',
'fh_ovarian',
'fh_pancreatic',
'fh_prostate',
'fh_skin',
'fh_stomach',
'fh_thyroid',
'fh_uterine',
'dx_abnormalities_gait_and_mobility',
'dx_any_cancer',
'dx_chronic_obstructive_pulmonary_disease',
'dx_cognitive_functions_and_awareness',
'dx_diabetes_mellitus_t1_or_12',
'dx_external_morbidity',
'dx_mood_affective_disorders',
'dx_smoking_or_nicotine_dependence',
'AgeAtVisit',
'AIDS',
'ALCOHOL',
'ANEMDEF',
'AUTOIMMUNE',
'BLDLOSS',
'CANCER_LEUK',
'CANCER_LYMPH',
'CANCER_METS',
'CANCER_NSITU',
'CANCER_SOLID',
'CBVD_POA',
'CBVD_SQLA',
'COAG',
'DEMENTIA',
'DEPRESS',
'DIAB_CX',
'DIAB_UNCX',
'DRUG_ABUSE',
'HF',
'HTN_CX',
'HTN_UNCX',
'LIVER_MLD',
'LIVER_SEV',
'LUNG_CHRONIC',
'NEURO_MOVT',
'NEURO_OTH',
'NEURO_SEIZ',
'OBESE',
'PARALYSIS',
'PERIVASC',
'PSYCHOSES',
'PULMCIRC',
'RENLFL_MOD',
'RENLFL_SEV',
'THYROID_HYPO',
'THYROID_OTH',
'ULCER_PEPTIC',
'VALVE',
'WGHTLOSS',
'cigna',
'comercial',
'medicare_advantage',
'managed_care',
'out_of_state',
'medicaid',
'medicare',
'CancerTypesNegativeInFamily_cnt',
'CancersTypesInFirstDegreeRelatives_cnt',
'CancersTypesInRelatives_cnt',
'Relatives_early_onset_cnt',
'Comorbidity_Cnt',
'OutOfState',
'OutOfCounty',
'Relatives_with_breast_cancer_cnt',
'CancerTypesNegativeInFamily',
'NoDxProblemsOrFamilyHistory',
'EPL_POV',
'EPL_UNEMP',
'EPL_PCI',
'EPL_NOHSDP',
'RPL_THEME1',
'EPL_AGE65',
'EPL_AGE17',
'EPL_DISABL',
'EPL_SNGPNT',
'EPL_MINRTY',
'EPL_LIMENG',
'EPL_MUNIT',
'EPL_MOBILE',
'EPL_CROWD',
'EPL_NOVEH',
'EPL_GROUPQ']

label_col = 'label'
X_all = df.loc[:,feature_names_all]

In [4]:
from sklearn.preprocessing import KBinsDiscretizer
discritizer = KBinsDiscretizer(4, strategy='uniform',  encode= 'ordinal').fit(X_all)
X_discrete = discritizer.transform(X_all)

In [5]:
data_for_structure_learning = pd.DataFrame(X_all, columns=feature_names_all)
data_for_structure_learning.loc[:, label_col] = df.loc[:,label_col ]


model = bn.structure_learning.fit(data_for_structure_learning, methodtype='tan', class_node = label_col)
# Plot detected DAG

# Compute edge strength using chi-square independence test
model1= bn.independence_test(model, data_for_structure_learning, alpha=0.1, prune=True)


# Parameter learning
model_paramterized = bn.parameter_learning.fit(model1, data_for_structure_learning)

# independed testing
bn_ind_test_results = model1['independence_test']



[bnlearn] >Computing best DAG using [tan]


Building tree:   0%|          | 0/6670.0 [00:00<?, ?it/s]

[bnlearn] >Compute edge strength with [chi_square]
[bnlearn] >Edge [RPL_THEME1 <-> AgeAtVisit] [P=0.441876] is excluded because it was not significant (P<0.10) with [chi_square]
[bnlearn] >Edge [RPL_THEME1 <-> CancersTypesInRelatives_cnt] [P=0.994678] is excluded because it was not significant (P<0.10) with [chi_square]
[bnlearn] >Edge [RPL_THEME1 <-> ph_cancer] [P=0.747851] is excluded because it was not significant (P<0.10) with [chi_square]
[bnlearn] >Edge [RPL_THEME1 <-> fh_lung] [P=0.676763] is excluded because it was not significant (P<0.10) with [chi_square]
[bnlearn] >Edge [RPL_THEME1 <-> fh_thyroid] [P=0.27394] is excluded because it was not significant (P<0.10) with [chi_square]
[bnlearn] >Edge [RPL_THEME1 <-> fh_pancreatic] [P=0.187752] is excluded because it was not significant (P<0.10) with [chi_square]
[bnlearn] >Edge [EPL_PCI <-> managed_care] [P=0.907647] is excluded because it was not significant (P<0.10) with [chi_square]
[bnlearn] >Edge [EPL_PCI <-> out_of_state] [P=

[bnlearn] >CPD of RPL_THEME1:
+--------------------+-----------------------+-----------------------+
| label              | label(0)              | label(1)              |
+--------------------+-----------------------+-----------------------+
| RPL_THEME1(0.001)  | 0.0038875755187811906 | 0.001746252925308608  |
+--------------------+-----------------------+-----------------------+
| RPL_THEME1(0.0017) | 0.0018673734985791713 | 0.0011165300033942507 |
+--------------------+-----------------------+-----------------------+
| RPL_THEME1(0.0046) | 0.0011939728251784984 | 0.001746252925308608  |
+--------------------+-----------------------+-----------------------+
| RPL_THEME1(0.0078) | 0.024089595720801386  | 0.014340711363595757  |
+--------------------+-----------------------+-----------------------+
| RPL_THEME1(0.0086) | 0.0032141748453805175 | 0.0011165300033942507 |
+--------------------+-----------------------+-----------------------+
| RPL_THEME1(0.01)   | 0.01735558898679465   | 

+-----------------+-----+----------------------+
| RPL_THEME1      | ... | RPL_THEME1(0.9706)   |
+-----------------+-----+----------------------+
| label           | ... | label(1)             |
+-----------------+-----+----------------------+
| EPL_PCI(0.0004) | ... | 0.003558718861209977 |
+-----------------+-----+----------------------+
| EPL_PCI(0.001)  | ... | 0.003558718861209977 |
+-----------------+-----+----------------------+
| EPL_PCI(0.0151) | ... | 0.003558718861209977 |
+-----------------+-----+----------------------+
| EPL_PCI(0.0167) | ... | 0.003558718861209977 |
+-----------------+-----+----------------------+
| EPL_PCI(0.0187) | ... | 0.003558718861209977 |
+-----------------+-----+----------------------+
| EPL_PCI(0.0245) | ... | 0.003558718861209977 |
+-----------------+-----+----------------------+
| EPL_PCI(0.0253) | ... | 0.003558718861209977 |
+-----------------+-----+----------------------+
| EPL_PCI(0.0345) | ... | 0.003558718861209977 |
+-----------------+-

...

+-------------------+-----+----------------------+
| RPL_THEME1        | ... | RPL_THEME1(0.9706)   |
+-------------------+-----+----------------------+
| label             | ... | label(1)             |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.0)    | ... | 0.009009009009009009 |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.0071) | ... | 0.009009009009009009 |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.008)  | ... | 0.009009009009009009 |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.0095) | ... | 0.009009009009009009 |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.0118) | ... | 0.009009009009009009 |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.0168) | ... | 0.009009009009009009 |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.0232) | ... | 0.009009009009009009 |
+-------------------+-----+----------------------+
| EPL_UNEMP(0.0315) | ... | 0.0

+--------------------+-----+----------------------+
| RPL_THEME1         | ... | RPL_THEME1(0.9706)   |
+--------------------+-----+----------------------+
| label              | ... | label(1)             |
+--------------------+-----+----------------------+
| EPL_MOBILE(0.0)    | ... | 0.006369426751592369 |
+--------------------+-----+----------------------+
| EPL_MOBILE(0.4231) | ... | 0.006369426751592369 |
+--------------------+-----+----------------------+
| EPL_MOBILE(0.436)  | ... | 0.006369426751592369 |
+--------------------+-----+----------------------+
| EPL_MOBILE(0.4524) | ... | 0.006369426751592369 |
+--------------------+-----+----------------------+
| EPL_MOBILE(0.4741) | ... | 0.006369426751592369 |
+--------------------+-----+----------------------+
| EPL_MOBILE(0.4938) | ... | 0.006369426751592369 |
+--------------------+-----+----------------------+
| EPL_MOBILE(0.5088) | ... | 0.006369426751592369 |
+--------------------+-----+----------------------+
| EPL_MOBILE

+-------------------+-----+----------------------+
| RPL_THEME1        | ... | RPL_THEME1(0.9706)   |
+-------------------+-----+----------------------+
| label             | ... | label(1)             |
+-------------------+-----+----------------------+
| EPL_CROWD(0.0)    | ... | 0.015873015873015886 |
+-------------------+-----+----------------------+
| EPL_CROWD(0.1771) | ... | 0.015873015873015886 |
+-------------------+-----+----------------------+
| EPL_CROWD(0.181)  | ... | 0.015873015873015886 |
+-------------------+-----+----------------------+
| EPL_CROWD(0.1883) | ... | 0.015873015873015886 |
+-------------------+-----+----------------------+
| EPL_CROWD(0.1999) | ... | 0.015873015873015886 |
+-------------------+-----+----------------------+
| EPL_CROWD(0.2179) | ... | 0.015873015873015886 |
+-------------------+-----+----------------------+
| EPL_CROWD(0.2418) | ... | 0.015873015873015886 |
+-------------------+-----+----------------------+
| EPL_CROWD(0.2699) | ... | 0.0

+--------------------+-----+----------------------+
| RPL_THEME1         | ... | RPL_THEME1(0.9706)   |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.0)    | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.3791) | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.44)   | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.5022) | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.5476) | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.5828) | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.6113) | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ(0.6342) | ... | 0.012381447638857943 |
+--------------------+-----+----------------------+
| EPL_GROUPQ

+------------------+----------------------+----------------------+
| label            | label(0)             | label(1)             |
+------------------+----------------------+----------------------+
| AgeAtVisit(50.0) | 0.026936026936026935 | 0.037783375314861464 |
+------------------+----------------------+----------------------+
| AgeAtVisit(51.0) | 0.032996632996632996 | 0.042821158690176324 |
+------------------+----------------------+----------------------+
| AgeAtVisit(52.0) | 0.03569023569023569  | 0.03904282115869018  |
+------------------+----------------------+----------------------+
| AgeAtVisit(53.0) | 0.03569023569023569  | 0.04408060453400504  |
+------------------+----------------------+----------------------+
| AgeAtVisit(54.0) | 0.03232323232323232  | 0.055415617128463476 |
+------------------+----------------------+----------------------+
| AgeAtVisit(55.0) | 0.03838383838383838  | 0.03841309823677582  |
+------------------+----------------------+-------------------

In [6]:

## calculates the markov blanket

bayesian_model = BayesianModel()
edge_list = zip(bn_ind_test_results.source, bn_ind_test_results.target)
for edge in edge_list:
    bayesian_model.add_edge(*edge)
markov_blanket = bayesian_model.get_markov_blanket(label_col)


markov_blanket


['EPL_AGE65',
 'RPL_THEME1',
 'Married',
 'dx_mood_affective_disorders',
 'DEPRESS',
 'EPL_PCI',
 'AgeAtVisit',
 'EPL_POV',
 'CancersTypesInRelatives_cnt',
 'EPL_MUNIT',
 'EPL_MINRTY',
 'ph_cancer',
 'medicaid',
 'EPL_NOVEH',
 'EPL_NOHSDP',
 'Single',
 'Comorbidity_Cnt',
 'fh_cancer',
 'out_of_state',
 'HTN_UNCX',
 'EPL_CROWD',
 'ANEMDEF',
 'OutOfCounty',
 'EPL_LIMENG',
 'CancerTypesNegativeInFamily_cnt',
 'EPL_AGE17',
 'fhneg_ovarian',
 'fh_breast',
 'EPL_MOBILE',
 'EPL_UNEMP']

In [7]:

print(F'number of feature prior to selection: {len(feature_names_all)} ')
print(F'number of features Selected by Markov Blanket: {len(markov_blanket)} ')

number of feature prior to selection: 115 
number of features Selected by Markov Blanket: 30 


In [147]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC


random_state =1231

X = df.loc[:, markov_blanket]
y = df.loc[:, label_col]

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# train Imputer
imputer = SimpleImputer(strategy='mean').fit(X)

## Build RF Estimator
#estimator =LogisticRegression(max_iter=5000, solver='liblinear', C=1, penalty='l2')

estimator =SVC(probability=True, C=.2,  kernel='rbf', random_state=random_state)
scaler =  StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


pipeline = Pipeline(steps=[('imp', imputer),('scale', StandardScaler()), ('estimator', estimator)]).fit(X_train,y_train)

# build cluster model
cluster_model = KMeans(3, random_state=random_state)
cluster_model = cluster_model.fit(X_train_scaled)

## Back Predict Data
p_train = pipeline.predict(X_train)
p_test = pipeline.predict(X_test)

p_prob_train = pipeline.predict_proba(X_train)[:, 1]
p_prob_test = pipeline.predict_proba(X_test)[:, 1]


print(F'n obs: {y.shape[0]}')
print(F'n positive lables: {y.sum()}')
print(F'label mean: {np.round(y.mean(),3)}')
print(F'n training obs: {y_train.shape[0]}')
print(F'n test obs: {y_test.shape[0]}')
print(F'train roc auc : {np.round(roc_auc_score(y_train, p_prob_train) , 3)}')
print(F'test roc auc : {np.round(roc_auc_score(y_test, p_prob_test), 3)}')

n obs: 2073
n positive lables: 1088
label mean: 0.525
n training obs: 1388
n test obs: 685
train roc auc : 0.677
test roc auc : 0.621


In [170]:
from sklearn.feature_selection import chi2
c2, imp_pval = selector = chi2(pd.DataFrame(X_discrete, columns=feature_names_all).loc[:,  markov_blanket], y)
c2

importance = pd.Series(c2, index=markov_blanket).sort_values(ascending=False).round(1)
importance

out_of_state                       52.3
AgeAtVisit                         31.1
medicaid                           25.4
OutOfCounty                        21.3
dx_mood_affective_disorders        21.3
DEPRESS                            18.1
ph_cancer                          16.0
ANEMDEF                            14.2
fhneg_ovarian                      11.9
fh_breast                          11.8
Comorbidity_Cnt                    11.7
HTN_UNCX                           11.4
fh_cancer                           9.4
EPL_UNEMP                           9.2
EPL_AGE17                           8.7
Single                              8.2
RPL_THEME1                          7.6
Married                             6.5
EPL_PCI                             5.6
EPL_POV                             4.5
EPL_NOVEH                           3.3
CancersTypesInRelatives_cnt         3.3
CancerTypesNegativeInFamily_cnt     2.6
EPL_MINRTY                          2.6
EPL_CROWD                           1.8


In [163]:
from plotly import express as px

group_col =  'Race'
source_col = 'case_control'



## calcuate train silhouette scores
train_clusters =  df.loc[X_train.index, group_col]
train_sil_scores = silhouette_samples(X_train_scaled, train_clusters )
print('train sil score',  train_sil_scores.mean())


## calcuate test silhouette scores
test_clusters =  df.loc[X_test.index, group_col]
test_sil_scores = silhouette_samples(X_test_scaled,test_clusters )
print('test sil score',  test_sil_scores.mean())



train_df = X_train.copy()
train_df.loc[:, 'Silhouette Score'] = train_sil_scores
train_df.loc[:, source_col] = df.loc[X_train.index, source_col]
train_df.loc[:, group_col] = train_clusters
train_df.loc[:, label_col] = y_train
train_df.loc[:, 'pred'] = p_prob_train
train_df.loc[:, 'pred_label'] = p_train


test_df = X_test.copy()
test_df.loc[:, 'Silhouette Score'] = test_sil_scores
test_df.loc[:, source_col] = df.loc[X_test.index, source_col]
test_df.loc[:,group_col] = test_clusters
test_df.loc[:, label_col] = y_test
test_df.loc[:, 'pred'] = p_prob_test
test_df.loc[:, 'pred_label'] = p_test




pred_prob = pipeline.predict_proba(df.loc[:, markov_blanket])[:, 1]
pred = pipeline.predict(df.loc[:, markov_blanket])
X_scaled = scaler.transform(df.loc[:, markov_blanket])
clusters = df.loc[:, group_col]
sil_scores = silhouette_samples(X_scaled,clusters  )


df.loc[:,'pred' ] = pred_prob
df.loc[:, 'pred_label'] = pred
df.loc[:, 'Silhouette Score'] = sil_scores

to_plot = df\
.sort_values(by=([group_col,'Silhouette Score']))\
.reset_index()

s_score = np.round(to_plot.loc[:, 'Silhouette Score'].mean(), 3)
title = F"Silhouette Analysis Showing Clustering with { group_col} <br>  Silhouette Score {s_score}"
fig = px.bar(to_plot, x ='Silhouette Score',  color=group_col, facet_col=source_col,  title=title, width=1000, height=400)
fig.update_traces(width=10)
fig.show()

train sil score -0.02651353528426348
test sil score 0.033948758729198815


In [12]:
px.histogram(to_plot,  x ='Silhouette Score', facet_col=group_col, color=source_col, title=title, width=1000, height=400)

In [13]:
to_plot.groupby([source_col, group_col]).agg({'Silhouette Score':'mean'}).sort_values(by = 'Silhouette Score', ascending=False)

Silhouette Score
case_control Race                                        
control      White-Caus                          0.025760
case         White-Caus                          0.015979
             Other or les frequent Race         -0.015169
             African-American                   -0.018541
control      Other or les frequent Race         -0.024576
             African-American                   -0.036254

In [14]:
## Decomposition (column compression and decompression)
decomp_model = PCA(20).fit(X_train_scaled )
train_comps = decomp_model.transform(X_train_scaled)
test_comps = decomp_model.transform(X_test_scaled)


train_comps_inv  = decomp_model.inverse_transform(train_comps)
test_comps_inv  = decomp_model.inverse_transform(test_comps)


X_comps = decomp_model.transform(X_scaled)
X_comps_inv = decomp_model.inverse_transform(X_comps)


In [15]:
feature_names = markov_blanket
def mae_all(arr1, arr2, names=X_test.columns):
    s = []
    p = []
    for i in np.arange(arr1.shape[1]):
        st = mean_absolute_error(arr1[:, i], arr2[:, i])
        pval = np.nan
        s.append(st)
        p.append(pval)
    return pd.DataFrame({'stat': s, 'pval': p}, index=names)
 
mae_dict = {}

for g in np.unique(df.loc[:, group_col]):
    temp_index =test_df.loc[test_df.loc[:, group_col] == g, :].index
    X_scaled_temp = pd.DataFrame(X_scaled, columns=markov_blanket, index=X.index).loc[temp_index,:]
    X_comps_inv_temp = pd.DataFrame(X_comps_inv, columns=markov_blanket, index=X.index).loc[temp_index,:]
    temp_df = mae_all( X_scaled_temp.values,X_comps_inv_temp.values )
    temp_df.loc[:, group_col] = g
    temp_df.index.name = 'Feature'
    mae_dict[g] =temp_df
    

mae_df = pd.concat(list(mae_dict.values()), axis=0).reset_index()

var_explained = decomp_model.explained_variance_ratio_.sum()
print(F"Percentage of Variance Explained by PCA {np.round(var_explained,3) *100 }%")

fig = px.bar(mae_df .reset_index(), y='Feature', x='stat', barmode="group", color=group_col ,
            width=1000, height=600, title='Error between Features and <br> PCA Inverse Representation of Features of Test Data')
fig.update_layout(yaxis={'categoryorder':'array', 'categoryarray':importance.index[::-1], })

fig.update_xaxes(title_text='Mean Squared Error')
fig.update_yaxes(title_text='Feature ranked by Model Importance')
fig.show()


Percentage of Variance Explained by PCA 94.0%


In [16]:
pd.pivot_table(df, index= 'case_control', columns='Race', values= 'CancerTypesNegativeInFamily_cnt').round(2)


Race,African-American,Other or les frequent Race,White-Caus
case_control,,,
case,0.25,0.09,0.18
control,0.28,0.13,0.23


In [73]:
agg = pd.pivot_table(df.reset_index(), index= 'CancerTypesNegativeInFamily_cnt', 
               columns='Race', 
               values= 'pid', 
               aggfunc='count').fillna(0)

agg_p = agg/agg.sum(axis=0)
to_plot  = agg_p.reset_index()\
.melt(id_vars='CancerTypesNegativeInFamily_cnt')\
.rename({'value': 'Proportion of Patients'}, axis=1)

agg

Race,African-American,Other or les frequent Race,White-Caus
CancerTypesNegativeInFamily_cnt,,,
0,668.0,57.0,1102.0
1,62.0,5.0,48.0
2,26.0,1.0,42.0
3,15.0,0.0,25.0
4,11.0,0.0,8.0
5,1.0,0.0,2.0


In [72]:
px.line(to_plot, x='CancerTypesNegativeInFamily_cnt', y = 'Proportion of Patients', color='Race',  markers=True)

In [17]:
pd.pivot_table(df, index= 'case_control', columns='Race', values= 'EPL_POV').round(2)

Race,African-American,Other or les frequent Race,White-Caus
case_control,,,
case,0.59,0.36,0.35
control,0.56,0.38,0.31


In [164]:
def update_performance_table(df, label_col, prediction_col, group, threshold):
    results = []
    y_true = df.loc[:, label_col]
    y_pred = df.loc[:, prediction_col]
    y_pred_label = pd.Series(y_pred> threshold).replace({True:1, False:0})
    groups =  np.unique(df.loc[:, group])
    groups_evaluated = []
    for g in groups:
        index = df.loc[:, group] == g
        y_true_temp = y_true.loc[index]
        y_pred_temp = y_pred.loc[index]
        y_pred_label_temp = y_pred_label.loc[index]
        try:
            tn, fp, fn, tp = confusion_matrix(y_true_temp, y_pred_label_temp).ravel()

            d = {'auc':  roc_auc_score(y_true_temp, y_pred_temp),
                 'f1' : f1_score(y_true_temp,y_pred_label_temp),
                 'precision' : precision_score(y_true_temp,y_pred_label_temp),
                 'recall': recall_score(y_true_temp, y_pred_label_temp),
                 'accuracy' : accuracy_score(y_true_temp, y_pred_label_temp),
                 'balanced_acc': balanced_accuracy_score(y_true_temp, y_pred_label_temp),
                 'TP': tp,
                 'FP': fp,
                 'TN': tn,
                 'FN': fn,
                 'weight': y_true_temp.shape[0],

                 }
            groups_evaluated.append(g)
            results.append(d)
        except:
            pass

    macro_avg = pd.DataFrame(results, index=groups_evaluated).mean(axis=0)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred_label).ravel()
    d = {'auc': roc_auc_score(y_true, y_pred),
         'f1': f1_score(y_true, y_pred_label),
         'precision': precision_score(y_true, y_pred_label),
         'recall': recall_score(y_true, y_pred_label),
         'accuracy': accuracy_score(y_true, y_pred_label),
         'balanced_acc': balanced_accuracy_score(y_true, y_pred_label),
         'TP': tp,
         'FP': fp,
         'TN': tn,
         'FN': fn,
         'weight': y_true.shape[0],

         }
    output = pd.DataFrame([d,  macro_avg.to_dict()], index=['MicroAverage', 'MacroAverage'
                           ]).round(3).reset_index()

    return output

print(F'Micro and Macro Average accross {group_col}')
update_performance_table(test_df, label_col, 'pred' , group_col, threshold=y.mean())

Micro and Macro Average accross Race


,index,auc,f1,precision,recall,accuracy,balanced_acc,TP,FP,TN,FN,weight
0,MicroAverage,0.621,0.611,0.622,0.600,0.599,0.598,216.0,131.000,194.000,144.0,685.000
1,MacroAverage,0.665,0.633,0.620,0.653,0.617,0.616,72.0,43.667,64.667,48.0,228.333


In [19]:
df.loc[:,group_col].value_counts().index

Index(['White-Caus', 'African-American', 'Other or les frequent Race'], dtype='object')

In [176]:
import plotly.graph_objs as go

prediction_col = 'pred'

train_df = X_train.copy()
train_df.loc[:, 'Silhouette Score'] = train_sil_scores
train_df.loc[:, 'Source'] ='Training Data (Original)'
train_df.loc[:, group_col] = train_clusters
train_df.loc[:, label_col] = y_train
train_df.loc[:, 'pred'] = p_prob_train
train_df.loc[:, 'pred_label'] = p_train


test_df = X_test.copy()
test_df.loc[:, 'Silhouette Score'] = test_sil_scores
test_df.loc[:, 'Source'] ='TestData'
test_df.loc[:,  group_col] = test_clusters
test_df.loc[:, label_col] = y_test
test_df.loc[:, 'pred'] = p_prob_test
test_df.loc[:, 'pred_label'] = p_test


def update_roc_auc(df, label_col, prediction_col, group):
    fig = go.Figure()
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )
    y_true = df.loc[:, label_col]
    y_pred = df.loc[:, prediction_col]
    groups =  np.unique(df.loc[:, group])
    for g in groups:
        index = df.loc[:, group] == g
        y_true_temp = y_true.loc[index]
        y_pred_temp = y_pred.loc[index]
        try:
            fpr, tpr, thresholds = roc_curve(y_true_temp, y_pred_temp)
            auc_score = roc_auc_score(y_true_temp, y_pred_temp)
            name = f"{g} (AUC={auc_score:.2f})"
            fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'))
        except:
            pass
    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=700, height=500
    )
    return fig

fig = update_roc_auc(test_df, label_col, prediction_col, group_col)

fig.update_layout(title={'text': F"Performance on Test Data (ROC AUC) by {group_col}"})

fig.show()

In [183]:
fig = update_roc_auc(test_df, label_col, prediction_col, "OutOfCounty")

fig.update_layout(title={'text': F"Performance on Test Data (ROC AUC) by OutOfCounty"})

fig.show()

In [211]:
px.box(test_df, x = 'label', y= 'pred', color = 'Race',facet_col="OutOfCounty", width=800,
      title='Model Has Degraded Performance When Patient Address are Outside of the Health Systems County <br> Regaurdless of Race')

In [172]:
px.box(test_df, x = 'label', y= 'pred', color = 'Race',facet_col="dx_mood_affective_disorders", width=1200)

out_of_state                       52.3
AgeAtVisit                         31.1
medicaid                           25.4
OutOfCounty                        21.3
dx_mood_affective_disorders        21.3
DEPRESS                            18.1
ph_cancer                          16.0
ANEMDEF                            14.2
fhneg_ovarian                      11.9
fh_breast                          11.8
Comorbidity_Cnt                    11.7
HTN_UNCX                           11.4
fh_cancer                           9.4
EPL_UNEMP                           9.2
EPL_AGE17                           8.7
Single                              8.2
RPL_THEME1                          7.6
Married                             6.5
EPL_PCI                             5.6
EPL_POV                             4.5
EPL_NOVEH                           3.3
CancersTypesInRelatives_cnt         3.3
CancerTypesNegativeInFamily_cnt     2.6
EPL_MINRTY                          2.6
EPL_CROWD                           1.8


In [29]:

p_prob_test =  estimator.predict_proba(X_test.loc[:, feature_names].values)[:, 1]
p_test = estimator.predict(X_test.loc[:, feature_names].values)

def get_classifcation_metrics(y_true, y_pred_label, y_pred_prob):
    d = {}
    try:
        d['ROC AUC'] = roc_auc_score(y_true,y_pred_prob )
    except ValueError:
       d['ROC AUC'] = np.nan 
    
    d['F1'] = f1_score(y_true, y_pred_label)
    d['Precision (PPV)'] = precision_score(y_true, y_pred_label)
    d['Recall (TPR)'] = recall_score(y_true, y_pred_label)
    d['Accuracy'] = accuracy_score(y_true, y_pred_label)
    d['Precision'] = precision_score(y_true, y_pred_label)
    d['Precision'] = precision_score(y_true, y_pred_label)
    d['Precision'] = precision_score(y_true, y_pred_label)
    try:
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred_label).ravel()
    except ValueError:
        tp =  confusion_matrix(y_true, y_pred_label).ravel()[0]
        fp, fn, tn = 0,0,0
    
    d['TP'] = tp
    d['FP'] = fp
    d['TN'] = tn
    d['FN'] = fn
    try:
        d['Specificity (TNR)'] = tn/(tn + fp)
    except ZeroDivisionError:
       d['Specificity (TNR)'] = np.nan 
    
    d['NPV'] = tn/(tp + fn)
    return d

def generate_performance_data(data, cluster_col, label_col, pred_col, pred_prob_col):
    for c in np.unique(data.loc[:, cluster_col]):
        temp_data = data.loc[data.loc[:, cluster_col] == c, :]
        y_true = temp_data.loc[ :, label_col]
        y_pred_label = temp_data.loc[ :, pred_col]
        y_pred_prob = temp_data.loc[:, pred_prob_col]
        results = get_classifcation_metrics(y_true, y_pred_label, y_pred_prob)
        results[cluster_col] = c
        yield results
        
performance_df = pd.DataFrame(list(generate_performance_data(test_df,group_col,  label_col,  'pred_label', 'pred', )))



In [32]:
performance_df.round(3)

,ROC AUC,F1,Precision (PPV),Recall (TPR),Accuracy,Precision,TP,FP,TN,FN,Specificity (TNR),NPV,Race
0,0.625,0.632,0.625,0.639,0.599,0.625,85,51,63,48,0.553,0.474,African-American
1,0.735,0.692,0.600,0.818,0.652,0.600,9,6,6,2,0.500,0.545,Other or les frequent Race
2,0.615,0.592,0.611,0.574,0.588,0.611,124,79,120,92,0.603,0.556,White-Caus


In [31]:
px.bar(performance_df, x=group_col, y='F1', barmode="group", width=600, height=500,
      title='F1 Score by Cluster')

In [194]:
importance

out_of_state                       52.3
AgeAtVisit                         31.1
medicaid                           25.4
OutOfCounty                        21.3
dx_mood_affective_disorders        21.3
DEPRESS                            18.1
ph_cancer                          16.0
ANEMDEF                            14.2
fhneg_ovarian                      11.9
fh_breast                          11.8
Comorbidity_Cnt                    11.7
HTN_UNCX                           11.4
fh_cancer                           9.4
EPL_UNEMP                           9.2
EPL_AGE17                           8.7
Single                              8.2
RPL_THEME1                          7.6
Married                             6.5
EPL_PCI                             5.6
EPL_POV                             4.5
EPL_NOVEH                           3.3
CancersTypesInRelatives_cnt         3.3
CancerTypesNegativeInFamily_cnt     2.6
EPL_MINRTY                          2.6
EPL_CROWD                           1.8


In [208]:
agg = pd.pivot_table(df.reset_index(),
               index= 'Comorbidity_Cnt', columns='Race', values= 'pid', aggfunc='count').fillna(0)


agg_p = agg/agg.sum(axis=0)
to_plot  = agg_p.reset_index()\
.melt(id_vars='Comorbidity_Cnt')\
.rename({'value': 'Proportion of Patients'}, axis=1)


In [209]:
px.line(to_plot, x='Comorbidity_Cnt', y = 'Proportion of Patients', color='Race',  markers=True)